In [4]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore")

In [5]:
gun_data = pd.read_csv('gun-violence-data-cleaned.csv')
gun_data = gun_data.drop(['Unnamed: 0'], axis=1)
# Kyle's code
# Replace date column with year, month, and day
gun_data.loc[:,'year'] = pd.DatetimeIndex(gun_data.loc[:,'date']).year
gun_data.loc[:,'month'] = pd.DatetimeIndex(gun_data.loc[:,'date']).month
gun_data.loc[:,'day'] = pd.DatetimeIndex(gun_data.loc[:,'date']).day
cols = gun_data.columns.tolist()
cols = cols[:1] + cols[-3:] + cols[2:-3]
gun_data = gun_data[cols]

In [6]:
# All CSVs
mental_health = pd.read_csv('EverytownNICSMentalHealth.csv')
gun_safety = pd.read_csv('GiffordsData.csv')
gun_ownership = pd.read_csv('Gun ownership.csv')
happiness = pd.read_csv('happiness.csv')
concealed_carry = pd.read_csv('ConcealedCarryPrices-1.csv')

In [7]:
# Happiness pre-processing
happiness['State'] = happiness['City']
for i in range(182):
    happiness['State'][i] = happiness['City'][i][-2:]

In [8]:
# Happiness pre-processing: converting two-letter abbrev. to state
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
#get full state names
happiness['State_full'] = happiness['State'].map(states)
#group by the full state name
happiness2 = happiness.groupby('State_full').count()['City'].to_frame()
happiness2 = happiness2.rename(index=str, columns={'City': 'Happiness Score'})

In [ ]:
#Kyle's Code
# Define a helper function to get a df of the people involved in an event
def get_people(row_data):  # A row from the main dataframe
    # Initialize data
    ages, genders, names, relationships, statuses, types = [], [], [], [], [], []
    
    # Regex the data
    if type(row_data['participant_age']) == str:
        ages = re.findall('(\d+)::(\d+)', row_data['participant_age'])
        ages = dict(ages)
    if type(row_data['participant_gender']) == str:
        genders = re.findall('(\d+)::(\w+)', row_data['participant_gender'])
        genders = dict(genders)
    if type(row_data['participant_name']) == str:
        names = re.findall('(\d+)::(\w+\s*\w*)', row_data['participant_name'])
        names = dict(names)
    if type(row_data['participant_relationship']) == str:
        relationships = re.findall('(\d+)::(\w+)', row_data['participant_relationship'])
        relationships = dict(relationships)
    if type(row_data['participant_status']) == str:
        statuses = re.findall('(\d+)::(\w+)', row_data['participant_status'])
        statuses = dict(statuses)
    if type(row_data['participant_type']) == str:
        types = re.findall('(\d+)::(\w+)', row_data['participant_type'])
        types = dict(types)
    
    # Get the number of people
    n_people = max(len(l) for l in [ages, genders, names, relationships, statuses, types])
    people_data = {'age': [], 'gender': [], 'name': [], 'relationship': [], 'status': [], 'type': []}
    for person in range(n_people):
        person = str(person)
        
        # Append the data
        if person in ages:
            people_data['age'].append(ages[person])
        else:
            people_data['age'].append(np.nan)
        if person in genders:
            people_data['gender'].append(genders[person])
        else:
            people_data['gender'].append('')
        if person in names:
            people_data['name'].append(names[person])
        else:
            people_data['name'].append('')
        if person in relationships:
            people_data['relationship'].append(relationships[person])
        else:
            people_data['relationship'].append('')
        if person in statuses:
            people_data['status'].append(statuses[person])
        else:
            people_data['status'].append('')
        if person in types:
            people_data['type'].append(types[person])
        else:
            people_data['type'].append('')
    
    # Extract the peoples' data
    people = pd.DataFrame.from_dict(people_data)
    
    return people

In [ ]:
# For each incident, generate column values for #female, #male, and various age groups
gun_data['n_female'] = gun_data['n_killed'].copy()
gun_data['n_male'] = gun_data['n_killed'].copy()
gun_data['n_unknown_gender'] = gun_data['n_killed'].copy()
gun_data['n_young'] = gun_data['n_killed'].copy()
gun_data['n_mid'] = gun_data['n_killed'].copy()
gun_data['n_old'] = gun_data['n_killed'].copy()
gun_data['n_unknown_age'] = gun_data['n_killed'].copy()
for i in range(len(gun_data)):
    if i % 100 == 0:
        print("processed:", i, "so far")
    people = get_people(gun_data.iloc[i])
    values = {'age': 100, 'gender': "Unknown"}
    people = people.fillna(value=values)
    gender_people = people.groupby('gender').count()['age'].to_frame().rename(index=str, columns={"age": "Count"})
    age_people = people.groupby('age').count()['gender'].to_frame().rename(index=str, columns={"gender": "Count"})
    age_people['Age'] = age_people.index
    try: 
        gun_data['n_female'][i] = gender_people.loc["Female"][0]
    except:
        gun_data['n_female'][i] = 0
    try: 
        gun_data['n_male'][i] = gender_people.loc["Male"][0]
    except:
        gun_data['n_male'][i] = 0
    try: 
        gun_data['n_unknown_gender'][i] = gender_people.loc["Unknown"][0]
    except:
        gun_data['n_unknown_gender'][i] = 0
    count_young = 0
    count_mid = 0
    count_old = 0
    count_unknown = 0
    for j in range(len(age_people)):
        age = float(age_people.iloc[j][1])
        count = age_people.iloc[j][0]
        
        if age < 25:
            count_young += count
        elif age < 45:
            count_mid += count
        elif age < 999:
            count_old += count
        else:
            count_unknown += count
    gun_data['n_young'][i] = count_young
    gun_data['n_mid'][i] = count_mid
    gun_data['n_old'][i] = count_old
    gun_data['n_unknown_gender'][i] = count_unknown

processed: 0 so far
processed: 100 so far
processed: 200 so far
processed: 500 so far
processed: 600 so far
processed: 700 so far
processed: 800 so far
processed: 900 so far
processed: 1000 so far
processed: 1100 so far
processed: 1200 so far
processed: 1300 so far
processed: 1400 so far
processed: 1500 so far
processed: 1600 so far
processed: 1700 so far
processed: 1800 so far
processed: 1900 so far
processed: 2000 so far
processed: 2100 so far
processed: 2400 so far
processed: 2500 so far
processed: 2600 so far
processed: 2700 so far
processed: 2800 so far


In [ ]:
# Merge Everything
merged1 = gun_data.merge(mental_health, left_on='state', right_on='State')
merged2 = merged1.merge(gun_safety, left_on='state', right_on='State')
merged3 = merged2.merge(gun_ownership, left_on='state', right_on='State')
merged4 = merged3.merge(concealed_carry, left_on='state', right_on='State')
merged5 = merged4.merge(happiness2, left_on='state', right_on='State_full')
merged5.columns